In [1]:
import json
import numpy as np
import pandas as pd
import os
import twitter

from twitter_auth import get_secret


In [2]:
f = "./woeids.json"

In [3]:
woeids = pd.read_json(f)

In [4]:
woeids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 7 columns):
country        467 non-null object
countryCode    466 non-null object
name           467 non-null object
parentid       467 non-null int64
placeType      467 non-null object
url            467 non-null object
woeid          467 non-null int64
dtypes: int64(2), object(5)
memory usage: 25.6+ KB


In [5]:
woeids.head()

,country,countryCode,name,parentid,placeType,url,woeid
0,,None,Worldwide,0,"{'code': 19, 'name': 'Supername'}",http://where.yahooapis.com/v1/place/1,1
1,Canada,CA,Winnipeg,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/2972,2972
2,Canada,CA,Ottawa,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3369,3369
3,Canada,CA,Quebec,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3444,3444
4,Canada,CA,Montreal,23424775,"{'code': 7, 'name': 'Town'}",http://where.yahooapis.com/v1/place/3534,3534


In [6]:
test = woeids.head()
# print(test.to_html())

In [7]:
woeids.loc[woeids['name'] == 'milwaukee']

,country,countryCode,name,parentid,placeType,url,woeid


In [8]:
def lookup(name, country='United States'):
    name = name.title()
    country = country.title()
    try:
        return woeids.loc[(woeids['country'] == country) & (woeids['name'] == name), 'woeid'].iloc[0]
    except IndexError: print("Not a valid country/name combo.")

In [9]:
lookup('united states')

23424977

In [10]:
twit_auth = get_secret()

In [11]:
consumer_key = twit_auth['CONSUMER_KEY']
consumer_secret = twit_auth['CONSUMER_SECRET']
access_token = twit_auth['ACCESS_TOKEN']
access_secret = twit_auth['ACCESS_SECRET']

In [12]:
api = twitter.Api(consumer_key, consumer_secret, access_token, access_secret)

In [13]:
def trending_by_geo(name, country="United States"):
    woeid = lookup(name, country)
    try:
        raw = [(x.name, x.volume) for x in api.GetTrendsWoeid(woeid)]
    except:
        print("Not a Twitter geographical category.")
    else:
        filtered = []
        for result in raw:
            if result[1] != None:
                filtered.append(result)
        results = pd.DataFrame(data=filtered, columns=["HashTag", "TweetVol"])
        results = results.sort_values("TweetVol", ascending=False)
        results.index = range(1, len(results)+1)
        results['TweetVol'] = results.TweetVol.apply(lambda x : "{:,}".format(x))
#         print("Top Trending HashTags in {}".format(name.title()))
        return results

In [14]:
result = trending_by_geo("Seattle")

In [17]:
print(result.to_json(orient=None))

{"HashTag":{"1":"Adele","2":"X \u00c6 A-12","3":"#NationalNursesDay","4":"Earl Thomas","5":"\"Ambassador\"","6":"#ThursdayThoughts","7":"Carville","8":"#supermoon","9":"#FelizJueves","10":"Sean Reed","11":"AP Exclusive","12":"Praying Mantis","13":"#InsideXbox","14":"#ConspiracyTheory","15":"#AhmaudAubrey","16":"Jason Whitlock"},"TweetVol":{"1":"794,251","2":"662,014","3":"130,072","4":"92,236","5":"43,024","6":"40,687","7":"26,325","8":"25,064","9":"22,731","10":"22,011","11":"14,382","12":"13,357","13":"13,026","14":"11,745","15":"11,659","16":"11,376"}}
